### LA CRISE ECONOMIQUE EN BELGIQUE ENTRE 1929 ET 1933

###Il est question de réaliser une étude comparative de deux journaux belges: Le Drapeau Rouge et La Libre Belgique; 
afin de voir les secteurs mis en avant pendant cette période

### I- Création de nos corpus d'analyse


###A partir des fichiers extraits de la base de données CAMILLE, nous allons créer nos deux grands corpus d'analyse

In [99]:
## IMPORTS
import os
import textract

#Le dossier Data1 est constitué des fichiers relatifs aux articles publiés dans le journal Le Drapeau Rouge pendant ladite période (1929-1933)

In [115]:
Data_path1 = '../data/Data1_Le Drapeau Rouge'
if not os.path.exists(Data_path1):
    os.mkdir(Data_path1)

#Le dossier Data2 quant à lui, est constitué d'articles publiés dans le journal La Libre Belgique

In [116]:
Data_path2 = '../data/Data2_La libre Belgique'
if not os.path.exists(Data_path2):
    os.mkdir(Data_path2)

##Création du corpus du journal Drapeau Rouge

In [117]:
with open("../data/Data1_Le Drapeau Rouge.txt", "w", encoding="utf-8") as output_file:
    for file in os.listdir(Data_path1):
        if file.endswith(".txt"):
            with open(os.path.join(Data_path1, file), "r", encoding="utf-8") as f:
                output_file.write(f.read())


##Création du corpus du journal La Libre Belgique

In [120]:
with open("../data/Data2_La libre Belgique.txt", "w", encoding="utf-8") as output_file:
    for file in os.listdir(Data_path2):
        if file.endswith(".txt"):
            with open(os.path.join(Data_path2, file), "r", encoding="utf-8") as f:
                output_file.write(f.read())
                

##Vérification du nombre de fichiers contenus dans chaque corpus

In [122]:
from collections import defaultdict
import os

import matplotlib.pyplot as plt
import numpy as np

In [126]:
Data_path1 = "../data/Data1_Le Drapeau Rouge/"
files = sorted(os.listdir(Data_path1))
len(files)

500

In [127]:
Data_path2 = "../data/Data2_La libre Belgique/"
files = sorted(os.listdir(Data_path2))
len(files)

500

#Nous venons ainsi de transformer les données que nous avons précedemment  extrait de la data base CAMILLE afin de constituer les corpus que nous allons ensuite exploiter tout au long de notre investigation.

## II- Extraction des keywords

##Dans cette étape, il est question d'extraire une liste de mots clés pour chacun de nos corpus afin de constater de potentielles similitudes entre les deux. 

In [128]:
#Importation de la librairie Yake
import os
import yake

In [129]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=100)
kw_extractor

##Extraction des mots clés de chacun des documents des corpus 1 et 2

In [138]:
##Corpus1
for f in sorted(os.listdir(Data_path1))[:500]:
    text1 = open(os.path.join(Data_path1, f), 'r', encoding="utf-8").read()
    keywords = kw_extractor.extract_keywords(text1)
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

KB_JB1051_1929-01-04_01-00002.txt mentions these keywords: Sénat décidèrent, jeunes métallurgistes, Comité Central, l'année politique, Vie congrès, jeune délégué, jeunes ouvriers, ans jour, prochain congrès, question linguistique, questions politiques, l'année écoulée, vieil ouvrier, XXe Siècle, buit heures, vie politique...
KB_JB1051_1929-01-04_01-00004.txt mentions these keywords: Parallèle suggestif, CUEfiâ Parallèle, nouveau film, cle films, DRAPEAST ROUGE, d'un film, Editions Socialistes, film ici, FILMS PARLANTS, film parlant, Film vient, cinéma soviétique, Fox Film, FILM FRANÇAIS, local habituel, grand film, film suédois, film entièrement, nouveaux films, domaine cle, film adapté, d'un nou, films d'TJmted, film nou-veaui, film paria-ut, films d'éducation, eau film, film intéressant, films d'après, films inspirés, film sportif, Parti Communiste...
KB_JB1051_1929-01-06_01-00002.txt mentions these keywords: chefs réformistes, paix industrielle, travailleurs belges, ouvriers belges,

In [139]:
#Corpus2
for f in sorted(os.listdir(Data_path2))[:500]:
    text2 = open(os.path.join(Data_path2, f), 'r', encoding="utf-8").read()
    keywords = kw_extractor.extract_keywords(text2)
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

KB_JB427_1929-01-26_01-00001.txt mentions these keywords: TTT DMl, SAINT BONAVENTURE, Saint Esprit, Action communiste, Chambre allemande, Comité spécial, c'est qu'il, vieux moteur, numéro jffidifciori, action anticommuniste, mouvement communiste, Parce qu'il, FERNAND MAILLAUD, Saint Bonaventuredans, Roger Bacon, philosophie médiévale...
KB_JB427_1929-01-30_01-00003.txt mentions these keywords: CADEAUX AUSSL, ijfj JANVIER, niUOITATIONS AITLUENT, Fernand Pacquement, Mme HANAU, suite d'un, l'Assemblée nationale, cours d'une, Jtoto Vpos, Lundi soir, note belge, EXTERIEUR FRANCE, d'un belge, France C'EST, Mme Miller, d'une crise, CIUDAD REAL, général Primo...
KB_JB427_1929-04-12_01-00002.txt mentions these keywords: Clayn inans, général Dawes, ous avez, tiore d'impôts, avez annoncé, désinvolture frisant, frisant l'incorrection, LISTE CATHOLIQUE, gouvernement d'action, corps d'armée, grande coalition, commissions cantonales, affaires étrangères, BUCEINGHAH Londres...
KB_JB427_1929-06-11_01-0

##Extraction de mots clés sur l'ensemble des documents de chaque corpus

In [144]:
# Extraire les mots clés du corpus 1
keywords1 = kw_extractor.extract_keywords(text1)
keywords1

[('DRAPEAU ROUGE', 0.009825864865179284),
 ('gouvernement', 0.010924012243516985),
 ('lutte', 0.015118792383451716),
 ('guerre', 0.018299273402754893),
 ('Secteur privé', 0.018693880081564092),
 ('politique', 0.019924442114906474),
 ('Hymans', 0.020342370861384827),
 ('ROUGE', 0.02317250908753769),
 ("C'est", 0.02354431207840583),
 ('travailleurs', 0.0248766483006499),
 ('Japon', 0.02500129620727912),
 ('gouvernement belge', 0.025073094913860793),
 ("d'une", 0.026678403636781152),
 ('plan', 0.027899926682999895),
 ('Secteur', 0.0290529880992989),
 ('Belgique', 0.02963437552675419),
 ('faut', 0.029859034155306653),
 ('Chine', 0.03161841424225701),
 ('privé', 0.036208142745491265),
 ('jour', 0.03897381843343045),
 ('DRAPEAU', 0.039674273170045),
 ('bourgeoisie', 0.044447896445256735),
 ('BELGE', 0.04574271524834081),
 ("l'économie", 0.0478938182638601),
 ('Macia', 0.05185031426295969),
 ('Tchang Kaï Chek', 0.05415447182867321),
 ("d'un gouvernement", 0.05581481065120737),
 ('pays', 0.056

In [151]:
# Ne garder que les bigrammes
kept = []
for kw, score in keywords1:
    words = kw.split()
    if len(words) == 2:
        kept.append(kw)
kept

['DRAPEAU ROUGE',
 'Secteur privé',
 'gouvernement belge',
 "d'un gouvernement",
 'GUERRE CONTINUE',
 "d'une lutte",
 "d'une politique",
 "l'armée rouge",
 "qu'il faut",
 'Tchang Kaï',
 'Kaï Chek',
 'partis politiques',
 'politique agressive',
 'préoccupations politiques',
 'Hymans continue',
 'Républiques Soviétiques',
 'unc économie',
 'Paul Hymans']

In [146]:
# Extraire les mots clés du corpus 2
keywords2 = kw_extractor.extract_keywords(text2)
keywords2

[('Bdltiou des journaui', 0.012481523713523174),
 ('France', 0.019984819357699198),
 ('Belgique', 0.020013028844137276),
 ('Libre Belgique', 0.024762236554906652),
 ('faire', 0.02961632736229865),
 ("d'une", 0.03234593559661862),
 ('Bruxelles', 0.03699189764864661),
 ("l'Allemagne", 0.03850667520046884),
 ('journal', 0.040589643732541966),
 ('point', 0.04110391826576606),
 ("qu'il", 0.041158040206621574),
 ('politique', 0.04475235087481609),
 ("l'année", 0.04661913033814739),
 ('Société des Nations', 0.04851524637802194),
 ('Xle Semaine Sociale', 0.05162934418205797),
 ('BON', 0.05200475236057137),
 ("c'est", 0.05243183694159262),
 ("d'un", 0.05634168740500791),
 ('Gorce', 0.05748945326387777),
 ('Semaine Sociale', 0.057651226333820305),
 ('économique', 0.05956414626497345),
 ('Ruysselede', 0.061568926347913375),
 ('catholique', 0.06273022730277486),
 ('Sociale Universitaire catholique', 0.06277421808641491),
 ('Dimanche', 0.06306931605631826),
 ('Montasrna-a', 0.06306931605631826),
 (

In [152]:
# Ne garder que les bigrammes
kept = []
for kw, score in keywords2:
    words = kw.split()
    if len(words) == 2:
        kept.append(kw)
kept

['Libre Belgique',
 'Semaine Sociale',
 "Légion d'honneur",
 'Xle Semaine',
 'président Roosevelt',
 'Patriote Illustré',
 'Universitaire catholique',
 'situation économique',
 'pris fin',
 "d'une politique",
 "d'une société",
 "qu'il fut",
 'faire partie',
 'second plan',
 'faire place',
 'numéro Edition',
 'paix monétaire']

### III-Constatation des déchets et nettoyage

##Dans cette partie, il est question d'épurer nos corpus. Pour ce faire, nous allons commencer par constater les déchets présents dans chacun d'eux, ensuite nous allons créer une liste de stopwords et enfin nous allons créer une fonction de nettoyage que nous allons appliquer sur chaque corpus.

1-Importation des librairies et stopwords

In [154]:
from collections import Counter
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kilia\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2- Ecrire tout le contenu des corpus dans un fichier temporaire et Constater des déchets

###Ecrire tout le contenu des corpus 1 et 2 dans un fichier temporaire

In [175]:
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'Data1_Le Drapeau Rouge'), 'w', encoding='utf-8') as f:
    f.write(' '.join( )

TypeError: str.join() takes no keyword arguments

In [165]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'Data2_La libre Belgique'), 'w', encoding='utf-8') as f:
    f.write(' '.join(Data))

##Constater les déchets

In [170]:
##Corpus 1
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(temp_path, f'Data1_Le Drapeau Rouge'), 'r', encoding='utf-8') as f:
    before = f.read()

before[:600]


'e m b r e   1 9 . - 3 .   I   , *   |   r f |   y   L E   D R A P E A U   R O U G E   " " " " " \' ?   \' * ■ ■ - \' " \' ■ ■   ^ y i n a ï t s   à   P a r i s   I   r u t a l   î l e   l \' A f e m a -   t é   d e s   N a t . \' s   e t   i j   ,   i n c m e n t ,   \\ f * -   s   i c i s t e   h i t l é r i e n   ( l i e s   »   d u   F u h   r >   p r o m i s   s \' é t a i t   t <   m e r n e m e n t   f r a n ç a   c o n d i t i o n   à   l \' o c t r o i   d e   1 \' e m p r . n t   d e s t i n é   é   d e s   w a u ï b   e t   à   p a y e r   l e s   7 5 6   m i l l i o n s   d e   D e v è z e , '

In [72]:
## Corpus Data2
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(txt_path2, file), 'r', encoding='utf-8') as f:
    before = f.read()

before[:10]

"1 ' ' ■ ' "

3- Liste de stop words

In [105]:
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "ville", "que", "voici", "parmi", "chaque", "leurs",
       "lequel", "alors", "laquelle", "trop", "fin", "tant", "ceux", "rien", "grandes",
       "tÃ©l", "dem", "trÃ¨s", "app", "part", "ans","cuis","tÃ©lÃ©ph","sociÃ©tÃ©", "rem","rue", 
       "gar", "grand", "ecrire", "place","monsieur", "quelques", "midi", "brux", "ecrire", "une", "sociÃ©tÃ©",
       "tél", "ecr", "téléph","vitae", "etc", "bur", "réf", "terr","très", "prés", "déjà", "pers",
       "effet", "fem","également", "mén", "vers", "toute", "trav","dès", "porte", "vis", "toujours",
       "démi", "bel", "celui", "jour", "chez", "mod", "samedi", "madame", "liv","tel", "bonne",
       "bon", "mois", "temps", "dimanche","saint", "bonnes", "peu", "près", "villa", "engage", "soir", "libre",
       "cherche", "vendre", "car", "demande", "etat","décès", "partie", "matin", "eau", "reprise","rossel", "premier",
       "première","maison", "jours","avant","cependant", "vue", "but","grande", "adresser", "dame"]
sw = set(sw)

4- Création d'une fonction de nettoyage pour chaque corpus

In [106]:
# Corpus Data1
def clean_text(txt_path1, folder=None):
    if folder is None:
        input_path = f"{txt_path1}.txt"
        output_path = f"{txt_path1}_clean.txt"
    else:
        input_path = f"{folder}/{txt_path1}.txt"
        output_path = f"{folder}/{txt_path1}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [81]:
# Corpus Data2
def clean_text(txt_path2, folder=None):
    if folder is None:
        input_path = f"{txt_path2}.txt"
        output_path = f"{txt_path2}_clean.txt"
    else:
        input_path = f"{folder}/{txt_path2}.txt"
        output_path = f"{folder}/{txt_path2}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

#Appliquer les fonctions à leur corpus respectif

In [107]:
clean_text (txt_path1, folder =None)

'Output has been written in ../data/Data1_Le Drapeau Rouge_clean.txt!'

In [82]:
clean_text (txt_path2, folder =None)

'Output has been written in ../data/Data2_La libre Belgique_clean.txt!'

In [108]:
with open(os.path.join(txt_path1, f'{txt_path1}_clean.txt'), 'r', encoding= 'utf-8') as f:
    after = f.read()

after[:500]

FileNotFoundError: [Errno 2] No such file or directory: '../data/Data1_Le Drapeau Rouge\\../data/Data1_Le Drapeau Rouge_clean.txt'